In [2]:
import pandas as pd
import re
import os
import time
import lib
from textwrap import dedent
from googletrans import Translator
from tqdm.auto import tqdm
translator = Translator()


In [3]:
df = pd.read_excel('./scrape_oci.xlsx', index_col=0)

In [4]:
def cleaning_korean(content):
    content = content.replace('\n\n','')
    content = ' '.join(content.split(' '))

    #클리닝1
    content = re.sub(r'... 기자', '', content)
    content = re.sub(r'...기자', '', content)

    #괄호안 제거 
    ''' 괄호안 내용 제거하기 '''
    pattern_bracket1 = r'\([^)]*\)'
    pattern_bracket2 = r'\{[^)]*\}'
    pattern_bracket3 = r'\<[^)]*\>'
    pattern_bracket4 = r'\[[^)]*\]'

    x = '이건 {괄호 안의 불필요한 정보를} 삭제하는 코드다.'
    content = re.sub(pattern=pattern_bracket1, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket2, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket3, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket4, repl='', string= content)
    content = ' '.join(content.split())
    #클리닝
    content = content.lower() #lower case
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    content = re.sub(pattern=pattern, repl='', string=content)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    content = re.sub(pattern=pattern, repl='', string=content)
    # 특수기호 특수
    content = re.sub('[-=+#/\?:^$@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》Δ△·“=◎<>▷]', '', content)
    content = re.sub('[一-龥]', '',content)
    #pattern = '[^\w\s]'         # 특수기호제거
    #content = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"]', '',str(content)) #remove punctuation
    content = re.sub(r"[^a-zA-Z0-9가-힣\.\s]","",content)
    content = re.sub(pattern=pattern, repl='', string=content)
    content = re.sub(r'\s+', ' ', content) #remove extra space
    content = re.sub(r'<[^>]+>','',content) #remove Html tags
    content = re.sub(r'\s+', ' ', content) #remove spaces
    content = re.sub(r"^\s+", '', content) #remove space from start
    content = re.sub(r'\s+$', '', content) #remove space from the end
    return content

In [5]:
all_contents = []
for i, row in df.iterrows():
    tit = row.Titles
    cot = row.Contents
    cot_all = str(tit) + '.' + str(cot)
    all_contents.append(cleaning_korean(cot_all))

In [6]:
%%time
contents_en = []
for x in tqdm(all_contents):
    try:
        src = "다음의 문장이 긍정,중립,부정 중 어느 카테고리에 속하는지 알려줘 " + '"' + str(x) + '"'
        src_en = translator.translate(src).text
        contents_en.append(src_en)
    except:
        contents_en.append('중립')

  0%|          | 0/6669 [00:00<?, ?it/s]

CPU times: total: 50.2 s
Wall time: 2h 3min 42s


In [8]:
df['contents_clean'] = all_contents
df['contents_en'] = contents_en

In [27]:
qq = []
for en in contents_en: 
    res = re.findall(r'\".*?\"', en)
    if res == []:
        qq.append('')
    else:
        qq.append(res[0].replace('"',''))

In [28]:
df['qa'] = qq

In [36]:
df

,Dates,Titles,Contents,contents_clean,contents_en,qa
0,2023.02.23 10:52,중국 신장지구에 지진 진도7.3,냉무,중국 신장지구에 지진 진도7.3.냉무,Please tell me which categories of the followi...,Earthquake in China's Xinjiang District 7.3. C...
1,2023.02.23 10:51,신한 [2],혼자 눌러대느라 애쓴다\n공매상환 하려면 못오르게 눌러야지\n그렇지만 그것도 곧한계...,신한 .혼자 눌러대느라 애쓴다 공매상환 하려면 못오르게 눌러야지 그렇지만 그것도 곧...,Let me know which categories of the following ...,
2,2023.02.23 10:50,애쓴다 비싸게,팔려고 저 매수호가 봐라 ㅋㅋ,애쓴다 비싸게.팔려고 저 매수호가 봐라,Tell me which categories of the following sent...,
3,2023.02.23 10:42,유안타 ㅉㅉ,이 타이밍에 매도를 하니까 메이저 증권사는 다 돈벌때 적자전환하지 ㅋㅋㅋ,유안타 .이 타이밍에 매도를 하니까 메이저 증권사는 다 돈벌때 적자전환하지,Let me know which categories of the following ...,
4,2023.02.23 10:33,프로그램 개 간보네 [1],높게 팔려고 ㅋㅋㅋ,프로그램 개 간보네 .높게 팔려고,Tell me which categories of the following sent...,
...,...,...,...,...,...,...
6664,2022.10.24 09:07,미국확장.. [2],굿..바로이거지.\n모레실적도 굿.\n달려보자,미국확장.. .굿..바로이거지. 모레실적도 굿. 달려보자,Please tell me which categories of the followi...,
6665,2022.10.23 23:10,내일 [2],장 괜찮을까요,내일 .장 괜찮을까요,Tell me which categories of the following sent...,Tomorrow. Is it okay?
6666,2022.10.23 18:51,이우현,3일남았다,이우현.3일남았다,Please tell me which category of the following...,Lee Woo -hyun. 3 There is a day left.
6667,2022.10.23 15:03,깔깔 마녀 등장!!!!!!!! [6],어이구 호9덜아 아직도 언니말안듣고 현실부정하는 못난이들 없ㅈ ㅔ~? ㅋ ㅔ~,깔깔 마녀 등장 .어이구 호9덜아 아직도 언니말안듣고 현실부정하는 못난이들 없,Let me know which categories of the following ...,


In [37]:
from easynmt import EasyNMT
model = EasyNMT('opus-mt')

In [42]:
resen = model.translate(all_contents,  target_lang='en')

Downloading:   0%|          | 0.00/796k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/281M [00:00<?, ?B/s]

If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.


OSError: Helsinki-NLP/opus-mt-dv-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)